# Sleep Scoring Model Multiple Files
Training a model on manual sleep scoring for multiple files.

In [3]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)


# Load data

In [4]:
input_dir = "C:\\dev\\play\\brainwave-data"


In [45]:
import os
import run_feature_pipeline
import manual_sleep_scoring_catboost_1 as helpers
errors = []

dfs = []

for root, dirs, files in os.walk(input_dir):
    for dir_name in dirs:
        input_file = os.path.join(root, dir_name, "raw.fif")
        scoring_file = os.path.join(root, dir_name, "raw.scorings.json")
        try:
            log("Processing file: " + input_file)
            if os.path.exists(input_file) and os.path.exists(scoring_file):
                scoring_df = helpers.load_scoring_file(scoring_file)
                yasa_df = run_feature_pipeline.cached_pipeline(log, input_file)
                assert(scoring_df.index.name)
                assert(yasa_df.index.name)
                joined = yasa_df.join(scoring_df)
                joined.reset_index(inplace=True)
                dfs.append(joined)
        except Exception as e:
            log("Error processing file: " + input_file)
            errors.append("Error processing file: " + input_file + " - " + str(e))
            log(e)

for err in errors:
    log(err)
    
yasa_df = pd.concat(dfs)
yasa_df.reset_index(inplace=True)
yasa_df.drop(columns=['index'], inplace=True)
yasa_df

,epoch,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,...,Predictions_has_microwaking_end,Predictions_Ambiguous Deep,Predictions_Deep,Predictions_Non-Deep,Predictions_Unsure,Predictions_Wake,Predictions_AnyDeep,scoredAt,ManualStage,tags
0,0,W,0.998098,0,2024-09-01 21:11:15.609066010+01:00,Fpz-M1,W,0.998098,0,1.924170e-09,...,0,0,0,0,0,1,0,2024-09-02T17:12:24.695Z,Wake,[Blinks]
1,1,W,0.998786,1,2024-09-01 21:11:45.609066010+01:00,Fpz-M1,W,0.998786,0,1.538265e-09,...,0,0,0,0,0,1,0,2024-09-02T17:12:24.791Z,Wake,[Blinks]
2,2,W,0.998694,2,2024-09-01 21:12:15.609066010+01:00,Fpz-M1,W,0.998694,0,3.222337e-09,...,0,0,0,0,0,1,0,2024-09-02T17:12:26.099Z,Wake,[Blinks]
3,3,W,0.997929,3,2024-09-01 21:12:45.609066010+01:00,Fpz-M1,W,0.997929,0,1.632114e-09,...,0,0,0,0,0,1,0,2024-09-02T17:12:57.334Z,Wake,[Blinks]
4,4,W,0.999127,4,2024-09-01 21:13:15.609066010+01:00,Fpz-M1,W,0.999127,0,1.673459e-09,...,0,0,0,0,0,1,0,2024-09-02T17:13:03.132Z,Wake,[Blinks]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,1062,W,0.910422,1062,2024-09-03 06:33:01.068267107+01:00,Fpz-M1,W,0.910422,0,1.166945e-08,...,0,0,0,0,0,1,0,2024-09-03T15:51:24.480Z,Noise,[]
2157,1063,W,0.916986,1063,2024-09-03 06:33:31.068267107+01:00,Fpz-M1,W,0.916986,0,1.011364e-08,...,0,0,0,0,0,1,0,2024-09-03T15:51:24.857Z,Noise,[]
2158,1064,W,0.895032,1064,2024-09-03 06:34:01.068267107+01:00,Fpz-M1,W,0.895032,0,9.848868e-09,...,0,0,0,0,0,1,0,2024-09-03T15:51:25.161Z,Noise,[]
2159,1065,W,0.872902,1065,2024-09-03 06:34:31.068267107+01:00,Fpz-M1,W,0.872902,0,1.144098e-08,...,0,0,0,0,0,1,0,2024-09-03T15:51:25.460Z,Noise,[]


In [34]:
errors

[]

In [17]:
yasa_df['tags'].value_counts()

tags
[]                                                 1841
[Possible non-wake disturbance]                     123
[Microwaking]                                        54
[MicrowakingComplete]                                48
[MicrowakingEnd]                                     34
[Weak deep]                                          19
[Blinks]                                             12
[Strong deep]                                        11
[Microwaking, MicrowakingEnd]                         7
[MicrowakingEnd, Possible non-wake disturbance]       3
[Unusual]                                             1
[Possible non-wake disturbance, Weak deep]            1
[Microwaking, Possible non-wake disturbance]          1
[Uniform]                                             1
[MicrowakingEnd, Microwaking]                         1
Name: count, dtype: int64

In [18]:
yasa_df['ManualStage'].value_counts()

ManualStage
Non-Deep          1310
Deep               304
Wake               300
Noise               86
Unsure              84
Ambiguous Deep      73
Name: count, dtype: int64

In [35]:
assert yasa_df.index.name

# Preparing data for model

In [61]:
import manual_sleep_scoring_catboost_1
prepared_df, X, y = manual_sleep_scoring_catboost_1.pipeline(yasa_df)

display(prepared_df)
display(X)
display(y)
display(y.columns.tolist())

C:\dev\play\brainwave-processor\models\manual_sleep_scoring_catboost_1\manual_sleep_scoring_catboost_1.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # X['ManualStage_strong_deep'] = X['tags'].apply(lambda x: 'Strong deep' in x)


,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_theta_p2min_norm,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Noise,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,0.281790,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,0.096633,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,0.052729,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,0.016031,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,-0.117325,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,1.166945e-08,0.954852,0.991652,0.130834,0.607724,0.644179,0.605366,0.351844,0.390320,1.205582,...,0.465007,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2157,1.011364e-08,0.947720,0.968089,0.148959,0.587216,0.524850,0.937471,0.311540,0.282771,0.636328,...,0.455670,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2158,9.848868e-09,0.941063,0.948977,0.138149,0.573423,0.504499,0.673968,0.278680,0.231414,0.635613,...,0.506617,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2159,1.144098e-08,0.936913,0.957363,0.148090,0.561054,0.477767,0.776224,0.256713,0.088185,0.653287,...,0.614377,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0


,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_at,Fpz-M1_eeg_at_c7min_norm,Fpz-M1_eeg_at_p2min_norm,Fpz-M1_eeg_auc,...,Fpz-M1_eeg_sigma_p2min_norm,Fpz-M1_eeg_skew,Fpz-M1_eeg_skew_c7min_norm,Fpz-M1_eeg_skew_p2min_norm,Fpz-M1_eeg_std,Fpz-M1_eeg_std_c7min_norm,Fpz-M1_eeg_std_p2min_norm,Fpz-M1_eeg_theta,Fpz-M1_eeg_theta_c7min_norm,Fpz-M1_eeg_theta_p2min_norm
0,1.924170e-09,0.674103,0.536489,0.026340,-0.560567,-0.410648,0.183803,-0.609504,-0.619848,0.291505,...,-0.286546,-0.041498,-0.292168,-0.168319,0.000061,0.408875,0.114821,0.143303,-0.031615,0.281790
1,1.538265e-09,0.742847,0.477407,0.019646,-0.566564,-0.444814,0.214660,-0.605466,-0.598287,0.330733,...,-0.314349,-1.054456,-0.295245,-0.300345,0.000072,0.457701,0.133877,0.091520,-0.061840,0.096633
2,3.222337e-09,0.804278,0.629601,0.015714,-0.569198,-0.469580,0.158740,-0.602967,-0.617150,0.842943,...,-0.298760,0.206473,-0.253102,-0.234790,0.000268,0.498781,0.366230,0.098994,-0.076986,0.052729
3,1.632114e-09,0.837591,0.583966,0.024473,-0.569084,-0.459612,0.269685,-0.596105,-0.587819,0.307789,...,-0.271386,-0.612750,-0.229889,-0.255400,0.000069,0.495865,0.310115,0.090746,-0.092663,0.016031
4,1.673459e-09,0.880257,0.564774,0.010777,-0.571818,-0.499329,0.156839,-0.593189,-0.597240,0.271290,...,-0.278903,-0.476831,-0.239482,-0.283770,0.000053,0.505479,0.303755,0.068711,-0.108333,-0.117325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,1.166945e-08,0.954852,0.991652,0.130834,0.607724,0.644179,0.605366,0.351844,0.390320,1.205582,...,0.893354,3.241335,-0.050473,0.073021,0.000442,0.844772,0.630973,0.216124,0.506553,0.465007
2157,1.011364e-08,0.947720,0.968089,0.148959,0.587216,0.524850,0.937471,0.311540,0.282771,0.636328,...,0.973514,-0.011915,-0.045118,0.070482,0.000105,0.746531,0.629507,0.158895,0.521069,0.455670
2158,9.848868e-09,0.941063,0.948977,0.138149,0.573423,0.504499,0.673968,0.278680,0.231414,0.635613,...,0.857279,-0.009031,-0.037225,0.075061,0.000105,0.638811,0.626474,0.204979,0.538211,0.506617
2159,1.144098e-08,0.936913,0.957363,0.148090,0.561054,0.477767,0.776224,0.256713,0.088185,0.653287,...,0.875710,-0.026285,-0.026923,0.074717,0.000109,0.512232,0.627268,0.190783,0.544987,0.614377


,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Noise,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...
2156,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2157,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2158,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0
2159,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0


['ManualStage_has_microwaking_start',
 'ManualStage_has_microwaking_end',
 'ManualStage_Ambiguous Deep',
 'ManualStage_Deep',
 'ManualStage_Noise',
 'ManualStage_Non-Deep',
 'ManualStage_Unsure',
 'ManualStage_Wake',
 'ManualStage_AnyDeep']

In [48]:
y['ManualStage_AnyDeep'].value_counts()

ManualStage_AnyDeep
0    1780
1     377
Name: count, dtype: int64

# Training a Catboost model

In [52]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    eval_metric='HammingLoss', 
    loss_function='MultiLogloss', 
    class_names=list(y.columns),
    iterations=1000,                # Number of boosting iterations
    learning_rate=0.03,             # Learning rate
    depth=6,                        # Depth of the tree
    l2_leaf_reg=3,                  # L2 regularization term on weights
    early_stopping_rounds=50        # Early stopping rounds
)
model.fit(X, y, verbose=100)

0:	learn: 0.0467213	total: 43.6ms	remaining: 43.6s
100:	learn: 0.0146809	total: 2.88s	remaining: 25.6s
200:	learn: 0.0060269	total: 5.49s	remaining: 21.8s
300:	learn: 0.0023695	total: 7.71s	remaining: 17.9s
400:	learn: 0.0011848	total: 9.94s	remaining: 14.8s
500:	learn: 0.0003091	total: 12.2s	remaining: 12.1s
600:	learn: 0.0000515	total: 14.4s	remaining: 9.58s
700:	learn: 0.0000000	total: 16.7s	remaining: 7.12s
800:	learn: 0.0000000	total: 18.9s	remaining: 4.71s
900:	learn: 0.0000000	total: 21.2s	remaining: 2.33s
999:	learn: 0.0000000	total: 23.5s	remaining: 0us


In [60]:
train_predict = model.predict(X)
train_predict_df = pd.DataFrame(train_predict, columns=y.columns)
train_predict_df

(2157, 9)

# Evaluating the model

In [55]:
rejoined = yasa_df.join(train_predict_df)
# cols_to_drop = [col for col in rejoined.columns if '_true' in col]
# rejoined.drop(columns=cols_to_drop, inplace=True)
rejoined

ValueError: columns overlap but no suffix specified: Index(['ManualStage_has_microwaking_start', 'ManualStage_has_microwaking_end'], dtype='object')

In [26]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions
y_pred_list = [(y_pred_prob > 0.5).astype(int) for y_pred_prob in train_predict]

# Calculate accuracy for each target column
accuracy_scores = {col: accuracy_score(y_train[col], y_pred) for col, y_pred in zip(y_train.columns, y_pred_list)}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')

ValueError: Found input variables with inconsistent numbers of samples: [872, 8]

In [32]:
display(y_train.shape)
display(train_predict.shape)
display(pd.DataFrame(train_predict, columns=y_train.columns))
train_predict_df = pd.DataFrame(train_predict, columns=y_train.columns)

(872, 8)

(872, 8)

,ManualStage_has_microwaking_start,ManualStage_has_microwaking_end,ManualStage_Ambiguous Deep,ManualStage_Deep,ManualStage_Non-Deep,ManualStage_Unsure,ManualStage_Wake,ManualStage_AnyDeep
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
867,0,0,0,0,1,0,0,0
868,0,0,0,1,0,0,0,1
869,0,0,0,0,0,0,1,0
870,0,0,0,0,1,0,0,0


In [33]:
from sklearn.metrics import accuracy_score

# Calculate accuracy for each target column
accuracy_scores = {col: accuracy_score(y_train[col], train_predict_df[col]) for col in y_train.columns}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')

Accuracy for ManualStage_has_microwaking_start: 1.0
Accuracy for ManualStage_has_microwaking_end: 1.0
Accuracy for ManualStage_Ambiguous Deep: 1.0
Accuracy for ManualStage_Deep: 1.0
Accuracy for ManualStage_Non-Deep: 0.9965596330275229
Accuracy for ManualStage_Unsure: 1.0
Accuracy for ManualStage_Wake: 1.0
Accuracy for ManualStage_AnyDeep: 0.9988532110091743


In [35]:
accuracy_scores = {col: accuracy_score(y_test[col], test_predict_df[col]) for col in y_train.columns}

# Print accuracy scores
for col, score in accuracy_scores.items():
    print(f'Accuracy for {col}: {score}')


Accuracy for ManualStage_has_microwaking_start: 0.9678899082568807
Accuracy for ManualStage_has_microwaking_end: 0.981651376146789
Accuracy for ManualStage_Ambiguous Deep: 0.9862385321100917
Accuracy for ManualStage_Deep: 0.9908256880733946
Accuracy for ManualStage_Non-Deep: 0.9770642201834863
Accuracy for ManualStage_Unsure: 0.981651376146789
Accuracy for ManualStage_Wake: 0.9908256880733946
Accuracy for ManualStage_AnyDeep: 0.9862385321100917


# Saving the model

In [62]:
import os
from datetime import datetime

# Get the current datetime and format it
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the directory if it doesn't exist
output_dir = "c:\\dev\\play\\BrainwaveModel"
os.makedirs(output_dir, exist_ok=True)

# Save the model with the current datetime in the filename
model_filename = f"sleep_scoring_catboost_model_{current_datetime}.cbm"
model.save_model(os.path.join(output_dir, model_filename))
model_filename